In [ ]:
import pymc as pm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import arviz as az
from scipy import stats

In [ ]:
# Load data from a CSV file
df = pd.read_csv('./student_scores.csv')
df
df.describe()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

sns.histplot(df, x='Hours', bins=10, ax=ax[0])
ax[0].set_title('Study Hours Hist')

sns.histplot(df, x='Scores', bins=10, ax=ax[1])
ax[1].set_title('Grades Score Hist')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate the correlation matrix for the data
df.corr()

In [ ]:
# Create a joint plot to visualize the relationship between 'Hours' and 'Scores' colored by 'Bins'
df['Bins'] = pd.cut(df['Hours'], bins=[0, 4, 7, float('inf')], labels=['low study', 'middle study', 'high study'])

sns.jointplot(data=df, x="Hours", y="Scores", hue="Bins")#, kind="kde")

plt.show()

In [ ]:
# Create a PyMC3 model
basic_model = pm.Model()
X = df['Hours'].values
Y = df['Scores'].values

# Define the Bayesian model
with basic_model:

    # Prior distributions for model parameters
    beta_0 = pm.Gamma('beta_0', alpha=20, beta=1)
    beta_1 = pm.Normal("beta_1", mu=5, sigma=2.5)
    sigma = pm.Gamma('sigma', alpha=2, beta=5)

    # Linear regression model equation
    mu = beta_0 + beta_1 * X

    # Likelihood of the observed data
    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=Y)

    # Perform Bayesian inference using MCMC sampling
    idata = pm.sample(return_inferencedata=True, draws=50000, tune=50000, chains=3)

In [ ]:
idata

In [ ]:
# Plot trace plots for the model parameters
pm.plot_trace(idata, compact=True)
plt.subplots_adjust(wspace=0.1, hspace=0.6)
plt.show()

In [ ]:
az.summary(idata, round_to=3)

In [ ]:
# Extract and display mean values of specific model parameters
mean_beta1, mean_beta0, mean_sigma = az.summary(idata, round_to=3)['mean'].values

In [ ]:
# Extract the median value for parameter 'beta_0' from the posterior samples
param_name = 'beta_0'
posterior_samples = idata.posterior[param_name].values.flatten()
median_value_b0 = np.median(posterior_samples)
print(f"Median value for parameter '{param_name}': {median_value_b0}")

In [ ]:
# Extract the median value for parameter 'beta_1' from the posterior samples
param_name = 'beta_1'
posterior_samples = idata.posterior[param_name].values.flatten()
median_value_b1 = np.median(posterior_samples)
print(f"Median value for parameter '{param_name}': {median_value_b1}")

In [ ]:
# Extract the median value for parameter 'sigma' from the posterior samples
param_name = 'sigma'
posterior_samples = idata.posterior[param_name].values.flatten()
median_value_sigma = np.median(posterior_samples)
print(f"Median value for parameter '{param_name}': {median_value_sigma}")

In [ ]:
# Plot posterior distributions for all model parameters
az.plot_posterior(idata)

In [ ]:
# Plot autocorrelation plots for all model parameters
az.plot_autocorr(idata, max_lag=100, combined = True)
plt.ylim(-1, 1)
plt.show()

In [ ]:
# Extract posterior samples for model parameters
posterior_samples = idata.posterior
beta_0_samples = posterior_samples['beta_0'].values
beta_1_samples = posterior_samples['beta_1'].values
sigma_samples = posterior_samples['sigma'].values

# Calculate mean values for model parameters after burn-in phase
beta_0_samples = beta_0_samples.mean(axis=0)[1000:10000]
beta_1_samples = beta_1_samples.mean(axis=0)[1000:10000]
sigma_samples = sigma_samples.mean(axis=0)[1000:10000]

print(len(beta_0_samples))
print(len(beta_1_samples))
print(len(sigma_samples))

In [ ]:
n_samples = 100000
X_range = np.linspace(0, 10, 100)

predictions = np.zeros((n_samples, len(X_range)))


indeces = np.array([k for k in range(len(sigma_samples)-1)])

# Generating predictions using posterior samples
for i in range(n_samples):

    index = np.random.choice(indeces)

    beta_0_sample = beta_0_samples[index]
    beta_1_sample = beta_1_samples[index]
    sigma_sample = sigma_samples[index]

    # Generating predictions based on the Bayesian linear regression model
    #y_sample = np.random.normal(loc=beta_0_sample + beta_1_sample * X_range, scale=sigma_sample)
    y_sample = np.random.normal(loc=median_value_b0 + median_value_b1 * X_range, scale=median_value_sigma)
    predictions[i, :] = y_sample

# Calculate the mean prediction and 95% credibility interval
mean_pred = predictions.mean(axis=0)
lower_bound = np.percentile(predictions, 2.5, axis=0)
upper_bound = np.percentile(predictions, 97.5, axis=0)


# Plot the MLE regression line, data points, and Bayesian linear regression results
slope, intercept, _, _, _ = stats.linregress(df['Hours'], df['Scores'])
plt.plot(X_range, intercept + slope * X_range, color='red', label='MLE Regression')


plt.scatter(df['Hours'], df['Scores'], color='blue', label='Data points')
plt.plot(X_range, mean_pred, color='blue', label='Mean prediction')
plt.fill_between(X_range, lower_bound, upper_bound, color='blue', alpha=0.1, label='95% credibility interval')
plt.xlabel('X')
plt.ylabel('Y')
plt.ylim(0.5, 99)
plt.xlim(0.5, 9.5)
plt.title('Bayesian Linear Regression')
plt.legend()
plt.show()

In [ ]:
# Sampling 100000 values from the Bayesian model and computing the 90% interval of the prediction
n_samples = 100000
X = 6.5

predictions = np.random.normal(loc=median_value_b0 + median_value_b1 * X, scale=median_value_sigma, size=n_samples)

pd.DataFrame(predictions).hist(bins=100)

In [ ]:
pd.DataFrame(predictions).quantile(0.95)

In [ ]:
pd.DataFrame(predictions).quantile(0.05)

In [ ]:
predictions.mean()

repeating the same process with non-informative priors

In [ ]:
uninf_model = pm.Model()
X = df['Hours'].values
Y = df['Scores'].values

with uninf_model:

    beta_0 = pm.Uniform('beta_0', lower=0, upper=100)
    beta_1 = pm.Normal("beta_1", mu=0, sigma=33)
    sigma = pm.Uniform('sigma', lower=0, upper=100)

    mu = beta_0 + beta_1 * X

    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=Y)

    idata = pm.sample(return_inferencedata=True, draws=100000, tune=100000, chains=3)


In [ ]:
pm.plot_trace(idata, compact=True)
plt.subplots_adjust(wspace=0.1, hspace=0.6)
plt.show()

In [ ]:
az.plot_autocorr(idata, max_lag=100, combined = True)
plt.ylim(-1, 1)
plt.show()

In [ ]:
az.summary(idata, round_to=3)

In [ ]:
mean_beta1, mean_beta0, mean_sigma = az.summary(idata, round_to=3)['mean'].values

In [ ]:
posterior_samples = idata.posterior
beta_0_samples = posterior_samples['beta_0'].values
beta_1_samples = posterior_samples['beta_1'].values
sigma_samples = posterior_samples['sigma'].values

beta_0_samples = beta_0_samples.mean(axis=0)[1000:10000]
beta_1_samples = beta_1_samples.mean(axis=0)[1000:10000]
sigma_samples = sigma_samples.mean(axis=0)[1000:10000]

print(len(beta_0_samples))
print(len(beta_1_samples))
print(len(sigma_samples))

In [ ]:
n_samples = 100000
X_range = np.linspace(0, 10, 100)

predictions = np.zeros((n_samples, len(X_range)))


indeces = np.array([k for k in range(len(sigma_samples)-1)])
for i in range(n_samples):

    index = np.random.choice(indeces)

    beta_0_sample = beta_0_samples[index]
    beta_1_sample = beta_1_samples[index]
    sigma_sample = sigma_samples[index]

    #y_sample = np.random.normal(loc=beta_0_sample + beta_1_sample * X_range, scale=sigma_sample)
    y_sample = np.random.normal(loc=mean_beta0 + mean_beta1 * X_range, scale=mean_sigma)
    predictions[i, :] = y_sample

mean_pred = predictions.mean(axis=0)
lower_bound = np.percentile(predictions, 2.5, axis=0)
upper_bound = np.percentile(predictions, 97.5, axis=0)

slope, intercept, _, _, _ = stats.linregress(df['Hours'], df['Scores'])
plt.plot(X_range, intercept + slope * X_range, color='red', label='MLE Regression')


plt.scatter(df['Hours'], df['Scores'], color='blue', label='Data points')
plt.plot(X_range, mean_pred, color='blue', label='Mean prediction')
plt.fill_between(X_range, lower_bound, upper_bound, color='blue', alpha=0.1, label='95% credibility interval')
plt.xlabel('X')
plt.ylabel('Y')
plt.ylim(0.5, 99)
plt.xlim(0.5, 9.5)
plt.title('Bayesian Linear Regression')
plt.legend()
plt.show()

In [ ]:
bad_model = pm.Model()
X = df['Hours'].values
Y = df['Scores'].values

with bad_model:

    beta_0 = pm.Gamma('beta_0', alpha=50, beta=0.1)
    beta_1 = pm.Normal("beta_1", mu=0, sigma=1)
    sigma = pm.Gamma('sigma', alpha=1, beta=2)

    mu = beta_0 + beta_1 * X

    Y_obs = pm.Normal("Y_obs", mu=mu, sigma=sigma, observed=Y)

    idata = pm.sample(return_inferencedata=True, draws=50000, tune=50000, chains=3)


In [ ]:
pm.plot_trace(idata, compact=True)
plt.subplots_adjust(wspace=0.1, hspace=0.6)
plt.show()

In [ ]:
az.plot_autocorr(idata, max_lag=100, combined = True)
plt.ylim(-1, 1)
plt.show()

In [ ]:
az.summary(idata, round_to=3)

In [ ]:
mean_beta1, mean_beta0, mean_sigma = az.summary(idata, round_to=3)['mean'].values

In [ ]:
posterior_samples = idata.posterior
beta_0_samples = posterior_samples['beta_0'].values
beta_1_samples = posterior_samples['beta_1'].values
sigma_samples = posterior_samples['sigma'].values

beta_0_samples = beta_0_samples.mean(axis=0)[1000:10000]
beta_1_samples = beta_1_samples.mean(axis=0)[1000:10000]
sigma_samples = sigma_samples.mean(axis=0)[1000:10000]

print(len(beta_0_samples))
print(len(beta_1_samples))
print(len(sigma_samples))

In [ ]:
n_samples = 100000
X_range = np.linspace(0, 10, 100)

predictions = np.zeros((n_samples, len(X_range)))


indeces = np.array([k for k in range(len(sigma_samples)-1)])
for i in range(n_samples):

    index = np.random.choice(indeces)

    beta_0_sample = beta_0_samples[index]
    beta_1_sample = beta_1_samples[index]
    sigma_sample = sigma_samples[index]

    #y_sample = np.random.normal(loc=beta_0_sample + beta_1_sample * X_range, scale=sigma_sample)
    y_sample = np.random.normal(loc=mean_beta0 + mean_beta1 * X_range, scale=mean_sigma	)
    predictions[i, :] = y_sample

mean_pred = predictions.mean(axis=0)
lower_bound = np.percentile(predictions, 2.5, axis=0)
upper_bound = np.percentile(predictions, 97.5, axis=0)

slope, intercept, _, _, _ = stats.linregress(df['Hours'], df['Scores'])
plt.plot(X_range, intercept + slope * X_range, color='red', label='MLE Regression')


plt.scatter(df['Hours'], df['Scores'], color='blue', label='Data points')
plt.plot(X_range, mean_pred, color='blue', label='Mean prediction')
plt.fill_between(X_range, lower_bound, upper_bound, color='blue', alpha=0.1, label='95% credibility interval')
plt.xlabel('X')
plt.ylabel('Y')
plt.ylim(0.5, 99)
plt.xlim(0.5, 9.5)
plt.title('Bayesian Linear Regression')
plt.legend()
plt.show()